In [121]:
import glob
import tensorflow as tf
from pathlib import Path
import math

def datasetImageFolder(start_path,split=None):
    filePath = getFilePaths(start_path,["jpg","png","jpeg"])
    splits =  list(set(map(lambda x: x[x.find('\\',len(start_path)-1 )+1:x.find('\\',x.find('\\',len(start_path)-1 )+1) ] , filePath)))
    
    allSplits = []
    for i in range(len(splits)):
        rs = list(map(lambda x : x if splits[i] in x else None, filePath))
        result = [x for x in rs if x]
        allSplits.append(result)
    
    datasets = dict()
    for i in range(len(allSplits)):
        images  = tf.data.Dataset.from_tensor_slices(allSplits[i])
        datasets[splits[i]] = images
    
    return datasets

def getFilePaths(start_path,extensions= None):
    paths = []
    if(extensions is not None):
        for extension in extensions:
            paths.extend(list(Path(start_path).rglob("*." + extension)))
    
    else:
        path.extend(list(Path(start_path).rglob()))
    
    return [str(x) for x in paths if x.is_file()]
    
    
def getLabelFromFilePathTF(file_path):
    splits = tf.strings.split(file_path,'\\')
    label = splits[len(splits)-2]
    return label

def getLabelFromFilePath(file_path):
    lastIndex = file_path.rfind('\\')
    firstIndex = file_path.rfind('\\',0,lastIndex)+1
    label =file_path[firstIndex:lastIndex]
    
    return label

def getAllLabels(start_path,generateUndefined=True):
    paths = getFilePaths(start_path,["jpg","png","jpeg"])
    distinctLabels = []
    if (generateUndefined):
        distinctLabels.append('Undefined')
    
    distinctLabels.extend(list(set(map(lambda x: getLabelFromFilePath(x) , paths))))
        
    return distinctLabels

def generateOneHotEncodeDict(labels):
    indices = [x for x in range(len(labels))]
    one_hots = tf.one_hot(indices,len(labels))
    
    one_hot_dict = dict()
    for i in range(len(one_hots)):
        one_hot_dict[labels[i]] = one_hots[i]
        
    return one_hot_dict
    
def generateBinaryEncoding(labels):
    indices = [x for x in range(len(labels))]
    n_bits = int(math.log(len(labels),2))
    binaryEncode = []
    for index in indices:
        encoding = []
        for i in range(n_bits+1):
            encoding.append( (index >> i) & 1)
        encoding.reverse()
        binaryEncode.append(encoding)
    
    print(labels)
    
    binaryEncodeTensors = [tf.convert_to_tensor(x,dtype=tf.uint8) for x in binaryEncode]
    
    binary_encode_dict = dict()
    for i in range(len(labels)):
        binary_encode_dict[labels[i]] = binaryEncodeTensors[i]
        
    return binary_encode_dict


    

In [122]:
ds = datasetImageFolder("Vgg\\data\\")

#print(datasetImageFolder("Vgg\\data\\"))
generateBinaryEncoding(getAllLabels("Vgg\\data\\",False))
dsTrain = ds['train']
dsTrain.map(lambda x  : tf.py_function(getLabelFromFilePathTF,[x],[tf.string,tf.string]))

dsTrain = dsTrain.shuffle(15000)

#for a in dsTrain.take(35):
   #print(a.numpy().decode("utf-8"))

['GLOSSY IBIS', 'RUFOUS KINGFISHER', 'BAY-BREASTED WARBLER', 'ANHINGA', 'CINNAMON TEAL', 'HOOPOES', 'MARABOU STORK', 'RED BROWED FINCH', 'GRAY CATBIRD', 'BLACK-NECKED GREBE', 'VIOLET GREEN SWALLOW', 'BAR-TAILED GODWIT', 'GOLDEN PHEASANT', 'AFRICAN CROWNED CRANE', 'MAGPIE GOOSE', 'AMERICAN GOLDFINCH', 'HOUSE SPARROW', 'CALIFORNIA QUAIL', 'NICOBAR PIGEON', 'TURQUOISE MOTMOT', 'PELICAN', 'ROYAL FLYCATCHER', 'SHOEBILL', 'CHUKAR PARTRIDGE', 'DOUBLE BARRED FINCH', 'CANARY', 'ENGGANO MYNA', 'GILA WOODPECKER', 'INDIAN PITTA', 'PURPLE MARTIN', 'STEAMER DUCK', 'CACTUS WREN', 'CALIFORNIA CONDOR', 'RAZORBILL', 'INDIGO BUNTING', 'BLACK SWAN', 'TRUMPTER SWAN', 'BELTED KINGFISHER', 'GOLDEN EAGLE', 'TASMANIAN HEN', 'EASTERN TOWEE', 'NORTHERN RED BISHOP', 'TEAL DUCK', 'SCARLET IBIS', 'SORA', 'ALEXANDRINE PARAKEET', 'BLACK-CAPPED CHICKADEE', 'MALACHITE KINGFISHER', 'HORNBILL', 'LARK BUNTING', 'PALILA', 'RED FACED WARBLER', 'TOUCHAN', 'SNOWY OWL', 'COMMON HOUSE MARTIN', 'PARADISE TANAGER', 'FLAME TANAGER

In [ ]:
100001101